<a href="https://colab.research.google.com/github/HDWilliams/User_Verification_HPE/blob/main/User_Verification_HPE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! git clone https://github.com/haofanwang/accurate-head-pose.git

Cloning into 'accurate-head-pose'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 42 (delta 3), reused 2 (delta 1), pack-reused 32
Unpacking objects: 100% (42/42), done.


In [ ]:
import sys
sys.path.append('/content/utils.py')
sys.path.append('/content/hopenet.py')

In [ ]:
#https://github.com/haofanwang/accurate-head-pose/blob/master/test_hopenet.py
import sys, os, argparse

import numpy as np
import cv2
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.backends.cudnn as cudnn
import torchvision
import torch.nn.functional as F

import importlib.util
import hopenet
import utils
from PIL import Image

def parse_args():
    """Parse input arguments."""
    parser = argparse.ArgumentParser(description='Head pose estimation using the Hopenet network.')
    parser.add_argument('--gpu', dest='gpu_id', help='GPU device id to use [0]',
            default=0, type=int)
    parser.add_argument('--data_dir', dest='data_dir', help='Directory path for data.',
          default='/AFLW2000/', type=str)
    parser.add_argument('--filename_list', dest='filename_list', help='Path to text file containing relative paths for every example.',
          default='/tools/AFLW2000_filename_filtered.txt', type=str)
    parser.add_argument('--snapshot', dest='snapshot', help='Name of model snapshot.',
          default='/output/snapshots/AFLW2000/_epoch_9.pkl', type=str)
    parser.add_argument('--batch_size', dest='batch_size', help='Batch size.',
          default=1, type=int)
    parser.add_argument('--save_viz', dest='save_viz', help='Save images with pose cube.',
          default=False, type=bool)
    parser.add_argument('--dataset', dest='dataset', help='Dataset type.', default='AFLW2000', type=str)

    args = parser.parse_args()

    return args

def run_pose_detection(model_path, img_PATH):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    """
    cudnn.enabled = True
    gpu = args.gpu_id
    snapshot_path = args.snapshot
    """
    print(hopenet)
    # ResNet50 structure
    model = hopenet.Multinet(torchvision.models.resnet.Bottleneck, [3, 4, 6, 3], 198)

    # Load snapshot
    saved_state_dict = torch.load(model_path, map_location=torch.device('cpu'))
    model.load_state_dict(saved_state_dict)


    transformations = transforms.Compose([transforms.Resize(224),
    transforms.CenterCrop(224), transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
    #test_loader = torch.utils.data.DataLoader(dataset=pose_dataset, batch_size=batch_size, num_workers=2)

    img = Image.open(img_PATH)
    img = transformations(img)
    img = img.unsqueeze(0)

    model.to(device)

    # Test the Model
    model.eval()  # Change model to 'eval' mode (BN uses moving mean/var).
    total = 0

    idx_tensor = [idx for idx in range(198)]
    idx_tensor = torch.FloatTensor(idx_tensor).to(device)

    l1loss = torch.nn.L1Loss(size_average=False)


    #for i, (images, labels, labels_0, labels_1, labels_2, labels_3, cont_labels, name) in enumerate(test_loader):
    img = Variable(img).to(device)
    
    yaw,yaw_1,yaw_2,yaw_3,yaw_4, pitch,pitch_1,pitch_2,pitch_3,pitch_4, roll,roll_1,roll_2,roll_3,roll_4 = model(img)

    # Binned predictions
    _, yaw_bpred = torch.max(yaw.data, 1)
    _, pitch_bpred = torch.max(pitch.data, 1)
    _, roll_bpred = torch.max(roll.data, 1)

    # Continuous predictions
    yaw_predicted = utils.softmax_temperature(yaw.data, 1)
    pitch_predicted = utils.softmax_temperature(pitch.data, 1)
    roll_predicted = utils.softmax_temperature(roll.data, 1)

    yaw_predicted = torch.sum(yaw_predicted * idx_tensor, 1).cpu() - 99
    pitch_predicted = torch.sum(pitch_predicted * idx_tensor, 1).cpu() - 99
    roll_predicted = torch.sum(roll_predicted * idx_tensor, 1).cpu() - 99
    print(f"Yaw: {yaw_predicted}")
    print(f"Pitch: {pitch_predicted}")
    print(f"Roll: {roll_predicted}")

    # Save first image in batch with pose cube or axis.
    cv2_img = cv2.imread(img_PATH)
        
    utils.plot_pose_cube(cv2_img, yaw_predicted[0], pitch_predicted[0], roll_predicted[0], size=100)
    utils.draw_axis(cv2_img, yaw_predicted[0], pitch_predicted[0], roll_predicted[0], tdx = 100, tdy= 100, size=100)
    cv2.imwrite(('/content/test.jpg'), cv2_img)

    return yaw_predicted, pitch_predicted, roll_predicted

In [ ]:
#testing test model function, 
#writes an image with yaw, pitch and roll estimation vectors and returns vectors for yaw, pitch and roll
run_pose_detection('/content/assets/AFLW2000.pkl', '/content/test_img/upper_right.jpg')

In [126]:
#import xml files for opencv face detection
! wget https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_profileface.xml

--2020-10-31 19:19:02--  https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_profileface.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 828514 (809K) [text/plain]
Saving to: ‘haarcascade_profileface.xml.1’

haarcascade_profile 100%[===================>] 809.10K  --.-KB/s    in 0.1s    

2020-10-31 19:19:02 (6.79 MB/s) - ‘haarcascade_profileface.xml.1’ saved [828514/828514]



In [133]:
from google.colab.patches import cv2_imshow
def detect_face(img_PATH):
  # Load the cascade
  face_cascade = cv2.CascadeClassifier('/content/assets/haarcascade_frontalface_default.xml')
  # Read the input image
  img = cv2.imread(img_PATH)
  # Convert into grayscale
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  # Detect faces
  faces = face_cascade.detectMultiScale(gray, 1.1, 4)

  if len(faces) > 1:
    print('Multiple faces detected')
    return False
  elif len(faces) < 1:
    print('No faces detected')
    return False
  # Draw rectangle around the faces
  for (x, y, w, h) in faces:
      cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
  # Display the output
  #cv2_imshow(img)
  cv2.waitKey()
  return True # TO DO may want to return face at some point as well

In [ ]:
#run get_face
print(detect_face('/content/test_img/front.jpg'))

In [147]:

def get_passcode_simp():
  #return random angles for yaw, pitch and roll between -45 and 45, and integer determing which axis the angle denotes
  #0 = yaw, 1 = pitch, 2 = roll
  interval = np.random.choice(2)
  if interval:
    pass_code = np.random.uniform(-30, -5)
  else:
    pass_code = np.random.uniform(5, 30)
  return np.int(pass_code), np.random.choice(3)

In [148]:
def verify_liveness_simp(initial_img_PATH, test_img_PATH, pass_code, index, margin_of_error = 1, model_PATH = '/content/assets/AFLW2000.pkl'):
  if detect_face(initial_img_PATH):
    #run pose detection and extract angles
    output = run_pose_detection(model_PATH, test_img_PATH) #TO DO, switch input variables for consistency
    test_code = np.int(output[index])
    if test_code < pass_code + margin_of_error and test_code > pass_code - margin_of_error:
      print("User Verfied...")
      return True
    else:
      print("Not Verified...")
      return False
  else:
    #no face or multiple faces detected
    return False

In [149]:
pass_code, index = get_passcode_simp()
print(pass_code, index)

5 1


In [152]:
initial_img_path = '/content/test_img/front.jpg'
test_img_PATH = '/content/test_img/front.jpg'
verify_liveness_simp(initial_img_path, test_img_PATH, pass_code, index)

<module 'hopenet' from '/content/hopenet.py'>


/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Yaw: tensor([-7.3137])
Pitch: tensor([3.1826])
Roll: tensor([2.0778])
Not Verified...


False